In [1]:
#basic imports
import pandas as pd
import numpy as np
import os

# env import
from env import host, user, password

In [2]:
def get_connection(db, username=user, host=host, password=password):
    '''
    Creates a connection URL
    '''
    return f'mysql+pymysql://{username}:{password}@{host}/{db}'
    
def new_zillow_data():
    '''
    Returns zillow into a dataframe
    '''
    sql_query = '''  SELECT *
    FROM properties_2017
    LEFT OUTER JOIN airconditioningtype 
    USING (airconditioningtypeid)
    LEFT OUTER JOIN architecturalstyletype
    USING (architecturalstyletypeid)
    LEFT OUTER JOIN buildingclasstype 
    USING (buildingclasstypeid)
    LEFT OUTER JOIN heatingorsystemtype
    USING (heatingorsystemtypeid)
    LEFT OUTER JOIN predictions_2017
    USING (id)
    INNER JOIN (
    SELECT id, MAX(transactiondate) as last_trans_date 
    FROM predictions_2017
    GROUP BY id
    ) predictions ON predictions.id = properties_2017.id AND predictions_2017.transactiondate = predictions.last_trans_date
    LEFT OUTER JOIN propertylandusetype
    USING(propertylandusetypeid)
    LEFT OUTER JOIN storytype
    ON storytype.storytypeid = properties_2017.storytypeid
    LEFT OUTER JOIN typeconstructiontype
    ON typeconstructiontype.typeconstructiontypeid = properties_2017.typeconstructiontypeid
    JOIN unique_properties
    ON unique_properties.parcelid = properties_2017.parcelid
    WHERE latitude IS NOT NULL and longitude IS NOT NULL; '''
    df = pd.read_sql(sql_query, get_connection('zillow'))
    return df 

def get_zillow_data():
    '''get connection, returns zillow into a dataframe and creates a csv for us'''
    if os.path.isfile('zillow.csv'):
        df = pd.read_csv('zillow.csv', index_col=0)
    else:
        df = new_zillow_data()
        df.to_csv('zillow.csv')
    return df

def drop_nulls(df, prop_req_col = .5 , prop_req_row = .5, inplace = True):
    threshold = int(prop_req_col * len(df.index)) 
    df.dropna(axis = 1, thresh = threshold, inplace = True)
    threshold = int(prop_req_row * len(df.columns)) 
    df.dropna(axis = 0, thresh = threshold, inplace = True)
    return df

In [3]:
df = get_zillow_data()

In [4]:
df.shape

(77614, 74)

In [5]:
df.head()

,propertylandusetypeid,id,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,parcelid,basementsqft,bathroomcnt,bedroomcnt,...,logerror,transactiondate,id,last_trans_date,propertylandusedesc,storytypeid,storydesc,typeconstructiontypeid,typeconstructiondesc,parcelid
0,269.0,0,NaN,NaN,NaN,NaN,10754147,NaN,0.0,0.0,...,0.025595,2017-01-01,0,2017-01-01,Planned Unit Development,NaN,None,NaN,None,10754147
1,261.0,1,NaN,NaN,NaN,NaN,10759547,NaN,0.0,0.0,...,0.055619,2017-01-01,1,2017-01-01,Single Family Residential,NaN,None,NaN,None,10759547
2,47.0,2,NaN,5.0,NaN,NaN,10843547,NaN,0.0,0.0,...,0.005383,2017-01-01,2,2017-01-01,Store/Office (Mixed Use),NaN,None,NaN,None,10843547
3,47.0,3,NaN,3.0,NaN,NaN,10859147,NaN,0.0,0.0,...,-0.103410,2017-01-01,3,2017-01-01,Store/Office (Mixed Use),NaN,None,NaN,None,10859147
4,31.0,4,NaN,4.0,NaN,NaN,10879947,NaN,0.0,0.0,...,0.006940,2017-01-01,4,2017-01-01,Commercial/Office/Residential Mixed Used,NaN,None,NaN,None,10879947


In [6]:
df = df.loc[:,~df.columns.duplicated()]

In [7]:
df.shape

(77614, 69)

In [8]:
df.head()

,propertylandusetypeid,id,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,parcelid,basementsqft,bathroomcnt,bedroomcnt,...,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,logerror,transactiondate,last_trans_date,propertylandusedesc,storydesc,typeconstructiondesc
0,269.0,0,NaN,NaN,NaN,NaN,10754147,NaN,0.0,0.0,...,None,None,None,None,0.025595,2017-01-01,2017-01-01,Planned Unit Development,None,None
1,261.0,1,NaN,NaN,NaN,NaN,10759547,NaN,0.0,0.0,...,None,None,None,None,0.055619,2017-01-01,2017-01-01,Single Family Residential,None,None
2,47.0,2,NaN,5.0,NaN,NaN,10843547,NaN,0.0,0.0,...,None,None,Specialized buildings that do not fit in any o...,None,0.005383,2017-01-01,2017-01-01,Store/Office (Mixed Use),None,None
3,47.0,3,NaN,3.0,NaN,NaN,10859147,NaN,0.0,0.0,...,None,None,Buildings having exterior walls built of a non...,None,-0.103410,2017-01-01,2017-01-01,Store/Office (Mixed Use),None,None
4,31.0,4,NaN,4.0,NaN,NaN,10879947,NaN,0.0,0.0,...,None,None,Buildings having wood or wood and steel frames,None,0.006940,2017-01-01,2017-01-01,Commercial/Office/Residential Mixed Used,None,None


In [9]:
single_use_codes = [261, 262, 263, 268, 273,275, 276, 279]
df = df[df['propertylandusetypeid'].isin(single_use_codes)]

In [10]:
df.shape

(57796, 69)

In [11]:
df = drop_nulls(df, prop_req_col = .5 , prop_req_row = .5, inplace = True)

In [12]:
df.shape

(57796, 35)

In [13]:
df.head()

,propertylandusetypeid,id,heatingorsystemtypeid,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,...,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,heatingorsystemdesc,logerror,transactiondate,last_trans_date,propertylandusedesc
1,261.0,1,NaN,10759547,0.0,0.0,NaN,NaN,NaN,NaN,...,27516.0,2015.0,27516.0,NaN,NaN,None,0.055619,2017-01-01,2017-01-01,Single Family Residential
15,261.0,15,NaN,11193347,0.0,0.0,NaN,NaN,NaN,NaN,...,10.0,2016.0,10.0,NaN,NaN,None,0.008669,2017-01-02,2017-01-02,Single Family Residential
16,261.0,16,NaN,11215747,0.0,0.0,NaN,NaN,NaN,NaN,...,10.0,2016.0,10.0,NaN,NaN,None,-0.021896,2017-01-02,2017-01-02,Single Family Residential
17,261.0,17,NaN,11229347,0.0,0.0,NaN,NaN,NaN,NaN,...,2108.0,2016.0,2108.0,174.21,NaN,None,-0.017167,2017-01-02,2017-01-02,Single Family Residential
20,261.0,20,2.0,11324547,2.0,4.0,8.0,2.0,3633.0,3633.0,...,296425.0,2016.0,74104.0,6941.39,NaN,Central,0.042463,2017-01-02,2017-01-02,Single Family Residential


In [14]:
dropcols = ['id','propertycountylandusecode','rawcensustractandblock','unitcnt']

In [15]:
df.drop(columns=dropcols)

,propertylandusetypeid,heatingorsystemtypeid,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,...,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,heatingorsystemdesc,logerror,transactiondate,last_trans_date,propertylandusedesc
1,261.0,NaN,10759547,0.0,0.0,NaN,NaN,NaN,NaN,6037.0,...,27516.0,2015.0,27516.0,NaN,NaN,None,0.055619,2017-01-01,2017-01-01,Single Family Residential
15,261.0,NaN,11193347,0.0,0.0,NaN,NaN,NaN,NaN,6037.0,...,10.0,2016.0,10.0,NaN,NaN,None,0.008669,2017-01-02,2017-01-02,Single Family Residential
16,261.0,NaN,11215747,0.0,0.0,NaN,NaN,NaN,NaN,6037.0,...,10.0,2016.0,10.0,NaN,NaN,None,-0.021896,2017-01-02,2017-01-02,Single Family Residential
17,261.0,NaN,11229347,0.0,0.0,NaN,NaN,NaN,NaN,6037.0,...,2108.0,2016.0,2108.0,174.21,NaN,None,-0.017167,2017-01-02,2017-01-02,Single Family Residential
20,261.0,2.0,11324547,2.0,4.0,8.0,2.0,3633.0,3633.0,6037.0,...,296425.0,2016.0,74104.0,6941.39,NaN,Central,0.042463,2017-01-02,2017-01-02,Single Family Residential
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77608,261.0,2.0,11212264,2.0,3.0,8.0,2.0,1600.0,1600.0,6037.0,...,180715.0,2016.0,42764.0,3677.13,6.037911e+13,Central,-0.002245,2017-09-20,2017-09-20,Single Family Residential
77609,261.0,2.0,11212539,3.0,4.0,8.0,3.0,1921.0,1921.0,6037.0,...,162019.0,2016.0,32453.0,2860.33,6.037911e+13,Central,0.020615,2017-09-20,2017-09-20,Single Family Residential
77610,261.0,2.0,11212639,3.0,4.0,8.0,3.0,1780.0,1780.0,6037.0,...,125923.0,2016.0,25179.0,2394.26,6.037911e+13,Central,0.013209,2017-09-21,2017-09-21,Single Family Residential
77611,261.0,2.0,11212962,2.0,3.0,6.0,2.0,1549.0,1549.0,6037.0,...,198988.0,2016.0,49747.0,3331.81,6.037911e+13,Central,0.037129,2017-09-21,2017-09-21,Single Family Residential


In [16]:
df.shape

(57796, 35)